# Homework 2 CSCI 4364/6364 Machine Learning

##**Adventures in Linear Regression**
v.20220920a

**Due Date: 10/4/2022, 23:59 ET**

---

**Purpose:**
This homework will familiarize you with linear regression using the [Prostate Cancer dataset](https://hastie.su.domains/ElemStatLearn/data.html). First, you’ll work with the least squares. Then you’ll investigate regression using L2 (Ridge) and L1 (Lasso) regularization. Finally, you will implement an iterative version of L2 Regularization using gradient descent.

**Note**: Besides part 3, you should implement your solution with the fundamental equations we discussed in class and in Hastie, chapter 3. *Only in part 3, you should use Scikit-Learn*.



---
**Submission Instructions:**
This assignment will be done entirely in this Colaboratory notebook, and you will submit your notebook via GWU blackboard. Please embed your code in code blocks and add in comments into the text blocks. 

**Grading on the notebook:**

Parts 1 - 4 of this notebook are worth 5% of the semester grade, where 3% is completion and full functionality, and 2% is based on comments and descriptions, and well-written and commented Python code, based on the coding standards. The notebook should be fully explained and work in its entirety when you submit it.

**Extra Credit!** Besides being a great learning experience about convex optimization, part 5 is **optional**, but worth up to 3% of the semester grade. 

**Coding Standards:**
Throughout this course, we will use Google’s Python Style Guide (https://google.github.io/styleguide/pyguide.html) as the coding standard for homework and project submission. A big part of machine learning in industry is applying good programming practices.


**Name:** Anulekha

**GW ID:** G40397446

In [ ]:
#@title Imports
##########################################################
# Always include all imports at the first executable cell.
##########################################################
from abc import ABC, abstractmethod # Abstract Base Classes for Python
import pandas as pd # Pandas dataframe libaries
import numpy as np # Numpy numerical computation library
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn import metrics # Used to compute metrics, such as the Area Under the Curve (AUC)
import matplotlib.pyplot as plt # Plotting library.
from typing import List, Tuple, Mapping # Common types for Python type definitions
from sklearn import linear_model


# Data Preperation & Feature Analysis (Prostate Cancer)



In [ ]:
#@title Load the Prostate Cancer dataset 
_SEED = 1223
random_state = np.random.RandomState(_SEED)
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/prostate.csv')
# Randomize the rows
df = df.sample(frac =  1, random_state=random_state)

In [ ]:
df.head()

In [ ]:
#@title Split into training and test set
# Following Hastie p.50, we create a training set of 67
split_index = 67
df_train = df.iloc[:split_index]
df_test = df.iloc[split_index:]

In [ ]:
#@title Display pair plots
sns.pairplot(df, kind="reg")
plt.show()

In [ ]:
#@title Split the labels and convert to numpy arrays
y_train = df_train['lpsa'].to_numpy() # independent variable is 'lpsa'
X_train = df_train.copy().drop(columns = ['lpsa']).to_numpy() # everything but the IV are the DV
y_test = df_test['lpsa'].to_numpy()
X_test = df_test.copy().drop(columns = ['lpsa']).to_numpy()

In [ ]:
class BaseLearningAlgorithm(ABC):
  """Base class for a Supervised Learning Algorithm."""

  @abstractmethod
  def train(self, X_train:np.array, y_train: np.array) -> None:
    """Trains a model from labels y and examples X."""

  @abstractmethod
  def predict(self, X_test: np.array) -> np.array:
    """Predicts on an unlabeled sample, X."""

  @property
  @abstractmethod
  def name(self) -> str:
    """Returns the name of the algorithm."""

# 1. Linear Regression with Least Squares
Implement a class called `BasicLeastSquaresRegression` that extends `BaseLearningAlgorithm` with “vanilla” least squares regression described in Hastie 3.2. Compute the $\boldsymbol{\beta}$ coefficient vector and solve for $\hat{y} $ and compute the performance result as the mean squared error on the test set. Use only numpy for your solution.





In [ ]:
class BasicLeastSquaresRegression(BaseLearningAlgorithm):

  def __init__(self):
    self._b = None

  def train(self, X_train:np.array, y_train: np.array) -> None:
    """Trains a model from labels y and examples X."""
    self._b = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)   

  def predict(self, X_test: np.array) -> np.array:
    """Predicts on an unlabeled sample, X."""
    y_hat = X_test.dot(self._b)
    return y_hat

  @property
  def name(self) -> str:
    """Returns the name of the algorithm."""
    return "ls"

  def MSE(self,y_test:np.array,y_hat:np.array):
    err_sq = (y_test - y_hat)**2
    mse = np.sum(err_sq)/y_test.shape[0]
    return mse

algo = BasicLeastSquaresRegression()
algo.train(X_train, y_train)
algo._b
y_hat = algo.predict(X_test)
print(y_hat)
algo.MSE(y_test,y_hat)


**Questions:**

**1.1** What MSE loss score do you get with Least Squares?



**1.2** What variables carry the greatest influence (i.e., are the most important) in the least squares regression?



**1.3** Does normalizing the data improve performance (lower MSE loss)? If so, why?


# 2. Linear Regression with L2 Regularization (Ridge)

Using the closed-form solution to Ridge Regression, implement a class called `RidgeRegression` that extends `BaseLearningAlgorithm` based on Hastie 3.41. Iterate through the regression penalty term, $\lambda$, and plot (a) MSE loss as a function of $\lambda$, and (b) each coefficient weight as a function of $\lambda$. Use only numpy and matplotlib for your solution.


In [ ]:
class RidgeRegression(BaseLearningAlgorithm):

  def __init__(self, lambdap):
    self._theta = None
    self._lambdap = lambdap


  def train(self, X_train:np.array, y_train: np.array) -> None:
    """Trains a model from labels y and examples X."""
    I = np.identity(X_train.shape[1])
    I[0][0] = 0
    self._theta = np.linalg.inv(X_train.T.dot(X_train) + self._lambdap * I).dot(X_train.T).dot(y_train)   

  
  def predict(self, X_test: np.array) -> np.array:
    """Predicts on an unlabeled sample, X."""
    y_hat_ridge = X_test.dot(self._theta)
    return y_hat_ridge

  @property
  def name(self) -> str:
    """Returns the name of the algorithm."""
    return "ridge_reg"

  def MSE(self,y_test:np.array,y_hat_ridge:np.array):
    err_sq = (y_test - y_hat_ridge)**2
    mse = np.sum(err_sq)/y_test.shape[0]
    print(mse)

ridge = RidgeRegression(lambdap=5)
ridge.train(X_train, y_train)
ridge._theta
print(ridge._lambdap)
y_hat_ridge = ridge.predict(X_test)
print(y_hat_ridge)
ridge.MSE(y_test,y_hat_ridge)

**Questions:**


**2.1** What $\lambda$ do you get the best performance (i.e., lowest MSE)?

The $\lambda$ with the best performance, or the lowest MSE, is 5. Anything below or higher results in a slightly higher MSE.

**2.2** Compare the results from part 1, when $\lambda = 0$. Why are the results similar or different?

The results are slightly different (~0.55 for $\lambda$=0 and ~0.54 for $\lambda$=5) because the added bias 

**2.3** Compare the weights with the weights from part 1? If you were to rank descending by the absolute value of the weights, how different is the ordering with part 1? Is the most important variable in part 1 the same as in part 2? If not, can you provide a reason?



# 3. Linear Regression with L1 Regularization (Lasso)

Unlike Ridge Regression, there is no closed-form solution for Lasso, meaning there is no normal equation we can solve to immediately get all of our ideal model parameters. While it can be solved by minimizing one coordinate a time using a technique called [Coordinate Descent](http://www.adeveloperdiary.com/data-science/machine-learning/introduction-to-coordinate-descent-using-least-squares-regression/), here you can implement your solution using Scikit-Learn Lasso inside a class called `LassoRegression` which also extends `BaseLearningAlgorithm`. Like part 2, iterate through the regression penalty term, $\alpha$ , and plot (a) MSE loss as a function of $\alpha$, and (b) each coefficient weight as a function of $\alpha$. 

**Note**: here we swap notation a little, replacing $\lambda$ with $\alpha$ to fit with the Scikit-Learn convention.




In [ ]:
# class LassoRegression(BaseLearningAlgorithm):

#   def __init__(self, ):
#     self._theta = None
#     self._lambdap = lambdap


#   def train(self, X_train:np.array, y_train: np.array) -> None:
#     """Trains a model from labels y and examples X."""
   
  
#   def predict(self, X_test: np.array) -> np.array:
#     """Predicts on an unlabeled sample, X."""
    

#   property
#   def name(self) -> str:
#     """Returns the name of the algorithm."""
#     return "lasso_reg"

In [ ]:
lasso = Lasso(alpha=1.0)
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test), lasso.score(X_train, y_train)

In [ ]:
lasso_yhat = lasso.coef_
lasso_yhat

In [ ]:
y_hat_lasso = np.asarray(lasso_yhat)
err_sq = (y_test - y_hat_lasso)**2
mse = np.sum(err_sq)/y_test.shape[0]
print(mse)

**Questions:**

**3.1** Under what conditions would you prefer L2 or L1 regression over vice versa? (Consider the discussion in Hastie 3.6.)

Both L1 and L2 regularization can help with multicollinearity.

L1 regularization can be used as a feature selection since it pins down coefficients to 0, while L2 constraints the coefficients and still keep all the variables. 

L1 is good for sparsity as it encourages 0 coefficients. It is useful for minimising the number of inputs.

L2, on the other hand, keeps all of the variables, therefore it is more useful when dealing with collinearity.

**3.2** Why do some coefficients become zero? Do you think this may be a method of subset selection as described in Hastie 3.3?

Some coefficients are set to zero in order to make the overall prediction better. Least squares method often has low bias but high variance. Setting coefficients to zero might increase bias slightly, but reduces variance, making for an overall better prediction. 

Yes. This is method of subset selection. 

**3.3** Which method performs better (i.e., has the lower MSE)?

L1 regression has the lower MSE and therefore performs better in this example.

**3.4** Comparing the relative ranking of the weights at the lowest MSE, do Ridge Regression and Lasso Regression “agree” on the most important weights?

**3.5** In your own words, define **bias** and **variance**. Describe how bias and variance affected the results of parts 1, 2, and 3.

Bias is how much the model's predictions differ from the target value compared to the training data. 

Variance describes how much the predictions will differ if a different training data was used. 

Added bias in the least squares method is 0. Increasing the bias by a $\lambda$ factor of 5 in the ridge regression in part 2 improves the model by reducing variance. This leads to a lower MSE score. 




# 4.  Iterative Optimization with Gradient Descent
Implement a class called `RidgeRegressionGradDescent` that extends `BaseLearningAlgorithm` that performs linear regression with L2 regularization using [gradient descent](https://www.deeplearningbook.org/contents/numerical.html), Goodfellow 4.5. Plot both training and test MSE loss result vs. iteration.  Iterate through the regression penalty term, $\lambda$ , and plot (a) MSE loss as a function of $\lambda$, and (b) each coefficient weight as a function of $\lambda$. Use only numpy and matplotlib for your solution.


**Questions:**

**4.1** How many iterations are required to converge, and are there any performance differences compared to part 2?

**4.2** Derive the gradient of L2 regularization loss. Encode your answer as [$\LaTeX$ equations](https://colab.research.google.com/github/bebi103a/bebi103a.github.io/blob/master/lessons/00/intro_to_latex.ipynb), and justify every step, please.

**4.3** Do you converge on the same minimum loss value? How do the coefficients compare to part 1?


#5. [OPTIONAL] Iterative Optimization using Coordinate Descent 

**Worth up to 3% of the semester grade.**

Review Hastie 3.8.6 and this [presentation](https://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/coordinate_descent.pdf) on coordinate descent. Using just numpy reimplement part 3 with coordinate descent. Like part 3, iterate through the regression penalty term, $\lambda$, and plot (a) MSE loss as a function of $\lambda$, and (b) each coefficient weight as a function of $\lambda$. Use only numpy and matplotlib for your solution.



**Questions:**

**5.1** How does your implementation compare with Scikit-Learn’s implementation in terms of minimum MSE loss and coefficients?

**5.2** Is the optimization surface convex or non-convex? 

**5.3** What makes the optimization surface *non-smooth* and how is coordinate descent able to overcome this problem?
